##1. **데이터 불러오기**:
구글드라이브 연동

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#crop_count = data_count - no_rec
import pandas as pd
front = [0]
slanted = [0]
no_rec = [0]
crop_count = pd.DataFrame((zip(front, no_rec, slanted)), columns =['Front', 'None-rec', 'Slanted'])

In [ ]:
crop_count

,Front,None-rec,Slanted
0,0,0,0


##2. **정면사진 검출 함수**
is_frontal_face(image_path)

In [ ]:
import math
# |v| = (벡터의 크기) 구하기
def dist(v):
    return math.sqrt(v[0]**2 + v[1]**2)

def innerAngle(v1, v2):
    # 벡터 v1, v2의 크기 구하기
    distA = dist(v1)
    distB = dist(v2)

    # 내적 1 (x1x2 + y1y2)
    ip = v1[0] * v2[0] + v1[1] * v2[1]

    # 내적 2 (|v1|*|v2|*cos x)
    ip2 = distA * distB

    # cos x값 구하기
    cost = ip / ip2

    # x값(라디안) 구하기 (cos 역함수)
    x = math.acos(cost)

    # x값을 x도로 변환하기
    degX = math.degrees(x)

    return (degX)

In [ ]:
import dlib
import math
import numpy as np
import cv2
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("/content/drive/MyDrive/머신러닝1/shape_predictor_68_face_landmarks.dat")  # dlib의 얼굴 랜드마크 파일 경로
faceCascade = cv2. CascadeClassifier ('/content/drive/MyDrive/머신러닝1/haarcascade_frontalface_default.txt')

def is_frontal_face(image_path):
    image = dlib.load_rgb_image(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces2 = faceCascade.detectMultiScale(gray, scaleFactor= 1.05, minNeighbors = 5, minSize = (100,100), flags = cv2.CASCADE_SCALE_IMAGE)
    if len(faces2)> 0:
      for (x, y, w, h) in faces2:
        dlib_rect = dlib.rectangle(int(x), int(y), int(x+w), int(y+h))
        landmarks = np.matrix([[p.x, p.y] for p in predictor(image, dlib_rect).parts()])
        landmarks_display = landmarks[0:68]

        right = np.squeeze(np.asarray(landmarks_display[16]))
        mid = np.squeeze(np.asarray(landmarks_display[28]))
        left = np.squeeze(np.asarray(landmarks_display[0]))
        left_wid=math.sqrt((left[0]-mid[0])**2+(left[1]-mid[1])**2)
        right_wid = math.sqrt((right[0]-mid[0])**2+(right[1]-mid[1])**2)
        ratio = (left_wid) / (right_wid)

        le_eye = np.squeeze(np.asarray(landmarks_display[0]))
        ri_eye = np.squeeze(np.asarray(landmarks_display[16]))
        nose = np.squeeze(np.asarray(landmarks_display[33]))

        le_vec = nose-le_eye
        ri_vec = nose-ri_eye

        angle = innerAngle(le_vec, ri_vec)

        if (angle > 90)& (angle < 151) :  #왼오 각도 구하기
          if (ratio > 0.8)& (ratio < 1.2) :  #좌우 비율 임계값 설정
            return True
        else:
          return False

        #if abs(slope) > 0.9
    else:
      #No_reg
      crop_count.iloc[0][2]+=1

##3. **모든 사진에서 오직 한 얼굴 검출하기**

In [ ]:
#data_Count 총 5000장
import pandas as pd
front = [0]
slanted = [0]
no_rec = [0]
data_count = pd.DataFrame((zip(front, no_rec, slanted)), columns =['Front', 'None-rec', 'Slanted'])

In [ ]:
#angle_df = pd.DataFrame(index = range(0,714), columns =['filename', 'Angle'])

In [ ]:
data_count

,Front,None-rec,Slanted
0,0,0,0


# 이미지 파일 읽기
        img_path = os.path.join(img_dir_path, filename)
        
        img = cv2.imread(img_path)

        # 이미지를 그레이스케일로 변환
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # 얼굴 검출
        faces = detector(gray, 1)

        # 검출된 얼굴 영역을 기반으로 얼굴 특징점을 인식
        for face in faces:
            landmarks = predictor(gray, face)

            # 검출된 얼굴 영역과 특징점을 그리기 위해 numpy 배열로 변환
            points = np.zeros((68, 2), dtype=int)
            for i in range(0, 68):
                points[i] = (landmarks.part(i).x, landmarks.part(i).y)

            # 얼굴 영역과 특징점을 이미지에 그림
            cv2.rectangle(img, (face.left(), face.top()), (face.right(), face.bottom()), (0, 255, 0), 2)
            for (x, y) in points:
                cv2.circle(img, (x, y), 2, (0, 0, 255), -1)

            # 인식된 얼굴 영역과 특징점이 그려진 이미지를 출력
            cv2_imshow(img)

##3. 특징점인식, 중앙얼굴, 정면 인식 함수

In [ ]:
import dlib
import math
import numpy as np
import cv2
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("/content/drive/MyDrive/머신러닝1/shape_predictor_68_face_landmarks.dat")  # dlib의 얼굴 랜드마크 파일 경로
faceCascade = cv2. CascadeClassifier ('/content/drive/MyDrive/머신러닝1/haarcascade_frontalface_default.txt')

from PIL import Image

def detect_person_in_center(image_dir):
  # 디렉토리 내 모든 이미지 파일에 대해 얼굴 특징점을 인식
  for filename in os.listdir(image_dir):
      #if filename.endswith(".jpg"):
      #or filename.endswith(".png") or filename.endswith(".JPG")or filename.endswith(".jpeg"):
      image_path = os.path.join(image_dir, filename)
      # Load the image
      image = dlib.load_rgb_image(image_path)

      # Convert the image to grayscale
      gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

      # Detect faces in the image
      faces = faceCascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

      # Check if any faces are detected
      if len(faces)> 0:
        #얼굴인식됨
        data_count.iloc[0][0]+=1
        # Find the center coordinates of the image
        height, width = image.shape[:2]
        center_x = width // 2
        center_y = height // 2

        # Calculate the distance between the center and each detected face
        distances = [abs((x + w // 2) - center_x) + abs((y + h // 2) - center_y) for (x, y, w, h) in faces]

        # Get the index of the face closest to the center
        closest_face_index = distances.index(min(distances))

        # Extract the coordinates of the closest face
        (x, y, w, h) = faces[closest_face_index]

        # Draw a rectangle around the closest face
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

        #정면 크롭
        image_plt = Image.open(image_path)
        croppedImage = image_plt.crop((x-3, y-3, 200, 200)) #위치정하기
        crop_path = os.path.join(crop_dir, 'crop_' + filename)
        #크롭 저장


        #croppedImage.save(crop_path)

      else:
        #No_reg 숫자 셈
        data_count.iloc[0][1]+=1
        #print("No faces detected in the image.")

def front_save(image_dir):
  # 디렉토리 내 모든 이미지 파일에 대해
  for filename in os.listdir(crop_dir):
        image_path = os.path.join(crop_dir, filename)
        image = Image.open(image_path)
        #정면사진만 추출
        result = is_frontal_face(image_path)
        if result:
          #정면임, Front
          crop_count.iloc[0][0]+=1
          #print("정면을 보고 있는 얼굴입니다.")
          # 이미지를 저장할 경로와 이름 지정
          save_path = os.path.join(save_dir, 'use_front_' + filename)
          # 이미지 저장


          #image.save(save_path)


          cv2.waitKey(0)
        else:
          #정면얼굴이 아님, Slanted
          crop_count.iloc[0][1]+=1
          #print("정면을 보고 있는 얼굴이 아닙니다.")


##4. 메인 프로그램

In [ ]:
import numpy as np
import cv2
import dlib

from google.colab.patches import cv2_imshow
import os

# 얼굴 검출을 위한 dlib 얼굴 인식 모델
detector = dlib.get_frontal_face_detector()


# 얼굴 특징점 검출을 위한 dlib 얼굴 인식 모델 MyDrive/파이널 과제 내분량/haarcascade_frontalface_default.txt')
predictor = dlib.shape_predictor ("/content/drive/MyDrive/머신러닝1/shape_predictor_68_face_landmarks.dat")

# 이미지가 저장된 디렉토리 경로
data_dir = "/content/drive/MyDrive/matlab/data_file/data"

# 크롭 이미지가 저장될 디렉토리 경로
crop_dir = "/content/drive/MyDrive/matlab/data_file/crop_data"

# (최종)정면 이미지가 저장될 디렉토리 경로
save_dir = "/content/drive/MyDrive/matlab/data_file/use_front_data"

In [ ]:
#인식,중앙에 있는 인물 사진 크롭 후 저장
detect_person_in_center(data_dir)

In [ ]:
front_save(crop_dir)

In [ ]:
data_count

,Front,None-rec,Slanted
0,5663,96,0


In [ ]:
crop_count

,Front,None-rec,Slanted
0,1531,4132,149
